In [ ]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
import jquantsapi
from datetime import datetime, timedelta
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

# 環境変数の読み込みとJ-Quants APIクライアントの初期化
load_dotenv()
my_mail_address = os.environ['JQUANTS_EMAIL']
my_password = os.environ['JQUANTS_PASSWORD']
cli = jquantsapi.Client(mail_address=my_mail_address, password=my_password)

class StockAnalyzer:
    def __init__(self, ticker_code):
        """
        株式分析クラスの初期化
        
        Parameters:
        ticker_code (str): 銘柄コード（例：'3917'）
        """
        self.ticker_code = ticker_code
        self.ticker_yahoo = f"{ticker_code}.T"  # Yahoo Finance用のティッカー
        self.jquants_client = cli
        self.analysis_results = {}
        
    def fetch_jquants_data(self):
        """J-Quantsからデータを取得"""
        try:
            # 企業情報の取得
            company_info = self.jquants_client.get_listed_info(code=self.ticker_code)
            
            # 日次株価データの取得（過去1年間）
            end_date = datetime.now()
            start_date = end_date - timedelta(days=365)
            
            daily_quotes = self.jquants_client.get_price_range(
                code=self.ticker_code,
                start_dt=start_date.strftime('%Y-%m-%d'),
                end_dt=end_date.strftime('%Y-%m-%d')
            )
            
            # 財務データの取得
            financial_data = self.jquants_client.get_statements_range(
                code=self.ticker_code,
                start_dt=start_date.strftime('%Y-%m-%d'),
                end_dt=end_date.strftime('%Y-%m-%d')
            )
            
            return {
                'company_info': company_info,
                'daily_quotes': daily_quotes,
                'financial_data': financial_data
            }
        except Exception as e:
            print(f"J-Quantsデータ取得エラー: {e}")
            return None
    
    def fetch_yfinance_data(self):
        """Yahoo Financeからデータを取得"""
        try:
            stock = yf.Ticker(self.ticker_yahoo)
            
            # 過去1年間の株価データ
            hist_data = stock.history(period="1y")
            
            # 企業情報
            info = stock.info
            
            # 財務諸表
            financials = stock.financials
            balance_sheet = stock.balance_sheet
            cashflow = stock.cashflow
            
            return {
                'historical': hist_data,
                'info': info,
                'financials': financials,
                'balance_sheet': balance_sheet,
                'cashflow': cashflow
            }
        except Exception as e:
            print(f"Yahoo Financeデータ取得エラー: {e}")
            return None
    
    def calculate_technical_indicators(self, price_data):
        """技術的指標の計算"""
        indicators = {}
        
        # 移動平均線
        indicators['SMA_20'] = price_data['Close'].rolling(window=20).mean().iloc[-1]
        indicators['SMA_50'] = price_data['Close'].rolling(window=50).mean().iloc[-1]
        indicators['SMA_200'] = price_data['Close'].rolling(window=200).mean().iloc[-1]
        
        # 現在価格
        current_price = price_data['Close'].iloc[-1]
        indicators['current_price'] = current_price
        
        # RSI（相対力指数）
        delta = price_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        indicators['RSI'] = (100 - (100 / (1 + rs))).iloc[-1]
        
        # MACD
        exp1 = price_data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = price_data['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1 - exp2
        signal = macd.ewm(span=9, adjust=False).mean()
        indicators['MACD'] = macd.iloc[-1]
        indicators['MACD_signal'] = signal.iloc[-1]
        indicators['MACD_histogram'] = (macd - signal).iloc[-1]
        
        # ボリンジャーバンド
        sma_20 = price_data['Close'].rolling(window=20).mean()
        std_20 = price_data['Close'].rolling(window=20).std()
        indicators['BB_upper'] = (sma_20 + (std_20 * 2)).iloc[-1]
        indicators['BB_lower'] = (sma_20 - (std_20 * 2)).iloc[-1]
        indicators['BB_position'] = (current_price - indicators['BB_lower']) / (indicators['BB_upper'] - indicators['BB_lower'])
        
        # 出来高分析
        indicators['volume_ratio'] = price_data['Volume'].iloc[-5:].mean() / price_data['Volume'].iloc[-30:].mean()
        
        return indicators
    
    def analyze_fundamentals(self, yf_data, jq_data=None):
        """ファンダメンタル分析"""
        fundamentals = {}
        
        try:
            info = yf_data['info']

            # 基本的な財務指標
            fundamentals['PER'] = info.get('trailingPE', np.nan)
            fundamentals['PBR'] = info.get('priceToBook', np.nan)
            fundamentals['dividend_yield'] = info.get('dividendYield', 0) * 100 if info.get('dividendYield') else 0
            fundamentals['market_cap'] = info.get('marketCap', np.nan)
            fundamentals['ROE'] = info.get('returnOnEquity', np.nan) * 100 if info.get('returnOnEquity') else np.nan
            fundamentals['profit_margin'] = info.get('profitMargins', np.nan) * 100 if info.get('profitMargins') else np.nan
            fundamentals['debt_to_equity'] = info.get('debtToEquity', np.nan)
            
            # 成長性指標
            fundamentals['revenue_growth'] = info.get('revenueGrowth', np.nan) * 100 if info.get('revenueGrowth') else np.nan
            fundamentals['earnings_growth'] = info.get('earningsGrowth', np.nan) * 100 if info.get('earningsGrowth') else np.nan
            
        except Exception as e:
            print(f"ファンダメンタル分析エラー: {e}")
            
        return fundamentals
    
    def calculate_sentiment_score(self, price_data):
        """市場センチメントスコアの計算"""
        sentiment = {}
        
        # 価格モメンタム
        sentiment['1week_return'] = ((price_data['Close'].iloc[-1] / price_data['Close'].iloc[-5]) - 1) * 100
        sentiment['1month_return'] = ((price_data['Close'].iloc[-1] / price_data['Close'].iloc[-20]) - 1) * 100
        sentiment['3month_return'] = ((price_data['Close'].iloc[-1] / price_data['Close'].iloc[-60]) - 1) * 100
        
        # ボラティリティ
        sentiment['volatility'] = price_data['Close'].pct_change().iloc[-30:].std() * np.sqrt(252) * 100
        
        # 52週高値・安値からの位置
        high_52w = price_data['Close'].iloc[-252:].max()
        low_52w = price_data['Close'].iloc[-252:].min()
        current = price_data['Close'].iloc[-1]
        sentiment['52w_position'] = ((current - low_52w) / (high_52w - low_52w)) * 100
        
        return sentiment
    
    def calculate_recommendation_score(self):
        """総合的な購入推奨度の計算"""
        scores = {
            'technical': 0,
            'fundamental': 0,
            'sentiment': 0
        }
        
        # 技術的指標スコア（40点満点）
        tech = self.analysis_results.get('technical', {})
        
        # RSIスコア
        if 30 < tech.get('RSI', 50) < 70:
            scores['technical'] += 10
        elif tech.get('RSI', 50) <= 30:
            scores['technical'] += 15  # 売られすぎ
        
        # 移動平均線スコア
        current = tech.get('current_price', 0)
        if current > tech.get('SMA_20', 0):
            scores['technical'] += 5
        if current > tech.get('SMA_50', 0):
            scores['technical'] += 5
        if current > tech.get('SMA_200', 0):
            scores['technical'] += 5
        
        # MACDスコア
        if tech.get('MACD_histogram', 0) > 0:
            scores['technical'] += 10
        
        # ボリンジャーバンド位置
        bb_pos = tech.get('BB_position', 0.5)
        if 0.2 < bb_pos < 0.8:
            scores['technical'] += 5
        
        # ファンダメンタルスコア（40点満点）
        fund = self.analysis_results.get('fundamentals', {})
        
        # PER評価
        per = fund.get('PER', 100)
        if 0 < per < 15:
            scores['fundamental'] += 15
        elif 15 <= per < 25:
            scores['fundamental'] += 10
        elif 25 <= per < 35:
            scores['fundamental'] += 5
        
        # ROE評価
        roe = fund.get('ROE', 0)
        if roe > 15:
            scores['fundamental'] += 10
        elif roe > 10:
            scores['fundamental'] += 7
        elif roe > 5:
            scores['fundamental'] += 3
        
        # 成長性評価
        rev_growth = fund.get('revenue_growth', 0)
        if rev_growth > 10:
            scores['fundamental'] += 10
        elif rev_growth > 5:
            scores['fundamental'] += 7
        elif rev_growth > 0:
            scores['fundamental'] += 3
        
        # 配当利回り
        div_yield = fund.get('dividend_yield', 0)
        if div_yield > 3:
            scores['fundamental'] += 5
        elif div_yield > 2:
            scores['fundamental'] += 3
        
        # センチメントスコア（20点満点）
        sent = self.analysis_results.get('sentiment', {})
        
        # モメンタム評価
        if sent.get('1month_return', 0) > 0:
            scores['sentiment'] += 7
        if sent.get('3month_return', 0) > 0:
            scores['sentiment'] += 7
        
        # 52週位置評価
        pos_52w = sent.get('52w_position', 50)
        if 30 < pos_52w < 70:
            scores['sentiment'] += 6
        
        # 総合スコア計算
        total_score = sum(scores.values())
        
        # 推奨度の判定
        if total_score >= 75:
            recommendation = "強い買い推奨"
        elif total_score >= 60:
            recommendation = "買い推奨"
        elif total_score >= 45:
            recommendation = "中立"
        elif total_score >= 30:
            recommendation = "売り検討"
        else:
            recommendation = "強い売り推奨"
        
        return {
            'scores': scores,
            'total_score': total_score,
            'max_score': 100,
            'percentage': (total_score / 100) * 100,
            'recommendation': recommendation
        }
    
    def generate_report(self):
        """分析レポートの生成"""
        print("=" * 80)
        print(f"株式分析レポート - 銘柄コード: {self.ticker_code}")
        print("=" * 80)
        
        # 技術的指標
        print("\n【技術的指標】")
        tech = self.analysis_results.get('technical', {})
        print(f"現在価格: ¥{tech.get('current_price', 'N/A'):,.0f}")
        print(f"RSI: {tech.get('RSI', 'N/A'):.2f}")
        print(f"MACD: {tech.get('MACD', 'N/A'):.2f}")
        print(f"20日移動平均: ¥{tech.get('SMA_20', 'N/A'):,.0f}")
        print(f"50日移動平均: ¥{tech.get('SMA_50', 'N/A'):,.0f}")
        print(f"200日移動平均: ¥{tech.get('SMA_200', 'N/A'):,.0f}")
        
        # ファンダメンタル指標
        print("\n【ファンダメンタル指標】")
        fund = self.analysis_results.get('fundamentals', {})
        print(f"PER: {fund.get('PER', 'N/A'):.2f}倍")
        print(f"PBR: {fund.get('PBR', 'N/A'):.2f}倍")
        print(f"ROE: {fund.get('ROE', 'N/A'):.2f}%")
        print(f"配当利回り: {fund.get('dividend_yield', 'N/A'):.2f}%")
        print(f"売上成長率: {fund.get('revenue_growth', 'N/A'):.2f}%")
        print(f"利益率: {fund.get('profit_margin', 'N/A'):.2f}%")
        
        # センチメント指標
        print("\n【市場センチメント】")
        sent = self.analysis_results.get('sentiment', {})
        print(f"1週間リターン: {sent.get('1week_return', 'N/A'):.2f}%")
        print(f"1ヶ月リターン: {sent.get('1month_return', 'N/A'):.2f}%")
        print(f"3ヶ月リターン: {sent.get('3month_return', 'N/A'):.2f}%")
        print(f"ボラティリティ（年率）: {sent.get('volatility', 'N/A'):.2f}%")
        print(f"52週レンジ内位置: {sent.get('52w_position', 'N/A'):.1f}%")
        
        # 推奨度
        print("\n【総合評価】")
        print("-" * 40)
        recommendation = self.analysis_results.get('recommendation', {})
        scores = recommendation.get('scores', {})
        
        print(f"技術的指標スコア: {scores.get('technical', 0)}/40点")
        print(f"ファンダメンタルスコア: {scores.get('fundamental', 0)}/40点")
        print(f"センチメントスコア: {scores.get('sentiment', 0)}/20点")
        print("-" * 40)
        print(f"総合スコア: {recommendation.get('total_score', 0)}/100点")
        print(f"推奨度: {recommendation.get('percentage', 0):.1f}%")
        print(f"\n【投資判断】: {recommendation.get('recommendation', 'N/A')}")
        
        # リスク要因
        print("\n【リスク要因】")
        self._print_risk_factors()
        
        # 投資戦略の提案
        print("\n【推奨投資戦略】")
        self._print_investment_strategy()
        
        print("\n" + "=" * 80)
    
    def _print_risk_factors(self):
        """リスク要因の分析と表示"""
        risks = []
        
        # ボラティリティリスク
        volatility = self.analysis_results.get('sentiment', {}).get('volatility', 0)
        if volatility > 40:
            risks.append(f"・高ボラティリティ（{volatility:.1f}%）- 価格変動リスクが高い")
        
        # バリュエーションリスク
        per = self.analysis_results.get('fundamentals', {}).get('PER', 0)
        if per > 30:
            risks.append(f"・高PER（{per:.1f}倍）- 割高感がある")
        
        # テクニカルリスク
        rsi = self.analysis_results.get('technical', {}).get('RSI', 50)
        if rsi > 70:
            risks.append(f"・RSI過熱（{rsi:.1f}）- 短期的な調整リスク")
        elif rsi < 30:
            risks.append(f"・RSI過度の売られ（{rsi:.1f}）- 下落トレンド継続リスク")
        
        # 財務リスク
        debt_to_equity = self.analysis_results.get('fundamentals', {}).get('debt_to_equity', 0)
        if debt_to_equity > 100:
            risks.append(f"・高レバレッジ（D/E比率: {debt_to_equity:.1f}%）")
        
        if risks:
            for risk in risks:
                print(risk)
        else:
            print("・現時点で特筆すべき大きなリスク要因は検出されていません")
    
    def _print_investment_strategy(self):
        """投資戦略の提案"""
        recommendation = self.analysis_results.get('recommendation', {})
        total_score = recommendation.get('total_score', 0)
        
        if total_score >= 75:
            print("・積極的な買い増しを検討")
            print("・目標保有期間: 中長期（6ヶ月以上）")
            print("・推奨投資配分: ポートフォリオの5-10%")
        elif total_score >= 60:
            print("・段階的な買い付けを推奨")
            print("・押し目買いのタイミングを待つ")
            print("・推奨投資配分: ポートフォリオの3-5%")
        elif total_score >= 45:
            print("・新規投資は見送り、既存ポジションは維持")
            print("・市場動向を注視し、明確なトレンド形成を待つ")
        else:
            print("・売却または空売りを検討")
            print("・既存ポジションの段階的な利益確定/損切りを推奨")
        
        # エントリー/エグジットポイントの提案
        tech = self.analysis_results.get('technical', {})
        current_price = tech.get('current_price', 0)
        sma_20 = tech.get('SMA_20', 0)
        bb_lower = tech.get('BB_lower', 0)
        bb_upper = tech.get('BB_upper', 0)
        
        if current_price > 0:
            print(f"\n・推奨エントリーポイント: ¥{bb_lower:,.0f} - ¥{sma_20:,.0f}")
            print(f"・利益確定目標: ¥{bb_upper:,.0f} (現在値から{((bb_upper/current_price - 1) * 100):.1f}%)")
            print(f"・損切りライン: ¥{(current_price * 0.95):,.0f} (現在値から-5%)")
    
    def run_analysis(self):
        """完全な分析の実行"""
        print("分析を開始します...")
        
        # データ取得
        print("Yahoo Financeからデータを取得中...")
        yf_data = self.fetch_yfinance_data()
        
        print("J-Quantsからデータを取得中...")
        jq_data = self.fetch_jquants_data()
        
        if yf_data is None:
            print("データ取得に失敗しました")
            return None
        
        # 各種分析の実行
        print("技術的指標を計算中...")
        self.analysis_results['technical'] = self.calculate_technical_indicators(yf_data['historical'])
        
        print("ファンダメンタル分析を実行中...")
        self.analysis_results['fundamentals'] = self.analyze_fundamentals(yf_data, jq_data)
        
        print("センチメント分析を実行中...")
        self.analysis_results['sentiment'] = self.calculate_sentiment_score(yf_data['historical'])
        
        print("総合評価を計算中...")
        self.analysis_results['recommendation'] = self.calculate_recommendation_score()
        
        # レポート生成
        self.generate_report()
        
        return self.analysis_results


# メイン実行部分
if __name__ == "__main__":
    # 銘柄コード3917の分析を実行
    analyzer = StockAnalyzer('3236')
    results = analyzer.run_analysis()
    
    # 結果をDataFrameとして保存（オプション）
    if results:
        # 分析結果をCSVファイルに保存
        summary_df = pd.DataFrame({
            '項目': ['総合スコア', '技術スコア', 'ファンダメンタルスコア', 'センチメントスコア', '推奨度'],
            '値': [
                results['recommendation']['total_score'],
                results['recommendation']['scores']['technical'],
                results['recommendation']['scores']['fundamental'],
                results['recommendation']['scores']['sentiment'],
                results['recommendation']['recommendation']
            ]
        })
        
        summary_df.to_csv(f'analysis_result_{analyzer.ticker_code}.csv', index=False, encoding='utf-8-sig')
        print(f"\n分析結果をanalysis_result_{analyzer.ticker_code}.csvに保存しました")